In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_core.tools import InjectedToolArg
import requests

c:\Ashuu\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [3]:
# Creating a tool
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [4]:
print(multiply.invoke({"a":10, "b":15}))
print(multiply.name)
print(multiply.description)

150
multiply
Given 2 numbers a and b this tool returns their product


# Tool Binding

In [5]:
llm_with_tool = llm.bind_tools([multiply])
print(llm_with_tool)

bound=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000015CC55B9780>, default_metadata=()) kwargs={'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'Given 2 numbers a and b this tool returns their product', 'parameters': {'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}}, 'required': ['a', 'b'], 'type': 'object'}}}]} config={} config_factories=[]


In [6]:
llm_with_tool.invoke("Hi how are you?")

AIMessage(content='I am doing well, thank you for asking. How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--01b0f27c-d0b7-406c-8055-1fe0a087cf3f-0', usage_metadata={'input_tokens': 25, 'output_tokens': 18, 'total_tokens': 43, 'input_token_details': {'cache_read': 0}})

In [7]:
result = llm_with_tool.invoke("Can you multiply 10 with 25")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 10.0, "b": 25.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--9fe0ab3c-6ef7-477a-b2dd-8b5b6c1f7409-0', tool_calls=[{'name': 'multiply', 'args': {'a': 10.0, 'b': 25.0}, 'id': 'ce85f1a2-d06b-4249-9f5b-e723854d4f0c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 30, 'output_tokens': 5, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}})

In [8]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 10.0, 'b': 25.0},
 'id': 'ce85f1a2-d06b-4249-9f5b-e723854d4f0c',
 'type': 'tool_call'}

In [9]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='250', name='multiply', tool_call_id='ce85f1a2-d06b-4249-9f5b-e723854d4f0c')

In [10]:
multiply.invoke(result.tool_calls[0]['args'])

250

In [11]:
query = HumanMessage('can you multiply 3 with 1000')

In [12]:
messages = [query]
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [13]:
result = llm_with_tool.invoke(messages)
messages.append(result)

In [14]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--f59998fb-9511-49fe-858b-d92a66013792-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': '419e37aa-c900-4cae-b795-cc44f0c05726', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}})]

In [15]:
tool_result = multiply.invoke(result.tool_calls[0])
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='419e37aa-c900-4cae-b795-cc44f0c05726')

In [16]:
messages.append(tool_result)

In [17]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--f59998fb-9511-49fe-858b-d92a66013792-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': '419e37aa-c900-4cae-b795-cc44f0c05726', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='3000', name='multiply', tool_call_id='419e37aa-c900-4cae-b795-cc44f0c05726')]

In [18]:
llm_with_tool.invoke(messages).content

'The product of 3 and 1000 is 3000.'

In [19]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'
  response = requests.get(url)
  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """
  return base_currency_value * conversion_rate


In [20]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [21]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1748995201,
 'time_last_update_utc': 'Wed, 04 Jun 2025 00:00:01 +0000',
 'time_next_update_unix': 1749081601,
 'time_next_update_utc': 'Thu, 05 Jun 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.7017}

In [22]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [23]:
# tool binding
llm = ChatGoogleGenerativeAI(model="model='gemini-2.0-flash'")

In [24]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [25]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [26]:
llm_with_tools

RunnableBinding(bound=ChatGoogleGenerativeAI(model="models/model='gemini-2.0-flash'", google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000015CC5942D70>, default_metadata=()), kwargs={'tools': [{'type': 'function', 'function': {'name': 'get_conversion_factor', 'description': 'This function fetches the currency conversion factor between a given base currency and a target currency', 'parameters': {'properties': {'base_currency': {'type': 'string'}, 'target_currency': {'type': 'string'}}, 'required': ['base_currency', 'target_currency'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'convert', 'description': 'given a currency conversion rate this function calculates the target currency value from a given base currency value', 'parameters': {'properties': {'base_currency_value': {'type': 'integer'}}, 'required': ['base_currency_value'], 'type': 'object'}}}]}, config={}

In [ ]:
ai_message =  llm_with_tools.invoke(messages).content

In [ ]:
messages.append(ai_message)
print(ai_message)

In [ ]:
ai_message.tool_calls

In [ ]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)

    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']

    # append this tool message to messages list
    messages.append(tool_message1)
    
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [ ]:
llm_with_tool.invoke(messages).content